# Rock Paper Scissors Image Classification

Nama : Agus Syahril Mubarok, Dicoding Indonesia

**1. Import Library TensorFlow, Keras dan Masukkan Dataset**

Masukan library model Deep Learning yang dipilih. Masukan layer, model, praproses gambar, dan optimasi pada metode Convolutional Neural Network. Dataset yang didapat adalah yang sudah disediakan oleh Dicoding.

In [ ]:
#import tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, Flatten, MaxPool2D

#Print tensorflow version
print(tf.__version__)

In [ ]:
#download dataset
!wget https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

 **2. Setup dan Import Library & Memisahkan Data Latih dan Validasi**

Masukan library model Deep Learning yang dipilih. Masukan layer, model, praproses gambar, dan optimasi pada metode Convolutional Neural Network.

Dataset yang didapat adalah yang sudah disediakan oleh Dicoding

Langkah selanjutnya adalah memisahkan data menjadi 2 bagian yaitu Data Train(Latih) dan Data Validasi dengan rasio yang diinginkan (60:40).

Dengan menguji model, kita dapat melihat kesalahan yang dibuat dan memperbaikinya sebelum mulai membawa model kita ke tahap selanjutnya

In [ ]:
#Install split folders
!pip install split_folders

In [ ]:
#Memisahkan data latih dan validasi
#Import zipfiles,os dan splitfolder
import zipfile,os
import splitfolders

local_zip = 'rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('rps')
zip_ref.close()

splitfolders.ratio('rps/rockpaperscissors/rps-cv-images', output="rps/rockpaperscissors/data", seed=1337, ratio=(.6, .4))
#splitfolders.fixed('rps/rockpaperscissors/rps-cv-images', output="rps/rockpaperscissors/data", seed=1337, fixed=(100, 100), oversample=False, group_prefix=None)


data_dir = 'rps/rockpaperscissors/data'
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'val')

os.listdir('rps/rockpaperscissors/data/train')
os.listdir('rps/rockpaperscissors/data/val')

In [ ]:
print(len(os.listdir('rps/rockpaperscissors/paper')))
print(len(os.listdir('rps/rockpaperscissors/scissors')))
print(len(os.listdir('rps/rockpaperscissors/rock')))

In [ ]:
train_paper_dir = os.path.join(train_dir, 'paper')
train_scissors_dir = os.path.join(train_dir, 'scissors')
train_rock_dir = os.path.join(train_dir, 'rock')

validation_paper_dir = os.path.join(validation_dir, 'paper')
validation_scissors_dir = os.path.join(validation_dir, 'scissors')
validation_rock_dir = os.path.join(validation_dir, 'rock')

In [ ]:
paper_imgs = os.listdir(train_paper_dir)
scissors_imgs = os.listdir(train_scissors_dir)
rock_imgs = os.listdir(train_rock_dir)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(22, 5))
for i, img_path in enumerate(paper_imgs[:5]):
  sp = plt.subplot(1, 5, i+1)
  img = mpimg.imread(os.path.join(train_paper_dir, img_path))
  plt.imshow(img)
plt.show()


In [ ]:
plt.figure(figsize=(22, 5))
for i, img_path in enumerate(scissors_imgs[:5]):
  sp = plt.subplot(1, 5, i+1)
  img = mpimg.imread(os.path.join(train_scissors_dir, img_path))
  plt.imshow(img)
plt.show()

In [ ]:
plt.figure(figsize=(22, 5))
for i, img_path in enumerate(rock_imgs[:5]):
  sp = plt.subplot(1, 5, i+1)
  img = mpimg.imread(os.path.join(train_rock_dir, img_path))
  plt.imshow(img)
plt.show()

**3. Augmentasi Data Gambar**

Augmentasi data gambar menggunakan metode ImageDataGenerator. Augmentasi data gambar berfungsi untuk menambahkan dataset gambar dengan cara memperbesar gambar, memotong gambar, dan memanipulasi gambar namun tanpa menghilangkan inti dari data tersebut.

In [ ]:
#Augmentasi data gambar
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.9999, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.9999)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (100, 100),
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = True
)

valid_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (100, 100),
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = True,
)

In [ ]:
train_generator.class_indices

In [ ]:
valid_generator.class_indices

**4. Melatih sebuah model Convolutional Neural Network**

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

**5. Kompilasi Model & Alur Pelatihan**

Metode selanjutnya menggunakan model yang akan dikompilasi. Penggunaan model ini agar menentukan optimasi dari model dataset tadi. Setelah menentukan pengoptimalan model dilanjutkan dengan penggunaan loss pada fungsi CNN dengan kelas dataset yang digunakan.

Untuk klasifikasi biner menggunakan binary_crossentropy dan untuk klasifikasi multi kelas menggunakan categorical_crossentropy dan kemudian menentukan metrik akurasi untuk mengevaluasi model yang dibuat.

In [ ]:
#Kompilasi model dan alur pelatihan

model.compile(loss = 'categorical_crossentropy',
              optimizer= 'RMSprop', 
              metrics= ['accuracy'])

In [ ]:
historyku = model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=20, 
          validation_data=valid_generator,
          validation_steps=5,
          verbose=2)

**6. Plot Kurva Data Latih dan Validasi**

Kemudian, kita tampilkan plot kurva hasil apakah data sudah cocok ataupun bisa terjadi overfitting dan underfitting.

In [ ]:
#Plot kurva data latih dan validasi
import matplotlib.pyplot as plt

acc = historyku.history['accuracy']
val_acc = historyku.history['val_accuracy']

loss = historyku.history['loss']
val_loss = historyku.history['val_loss']

plt.plot(acc, color='blue')
plt.plot(val_acc, color='red')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(loss, color='blue')
plt.plot(val_loss, color='red')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show

**7. Memprediksi Model Data yang ada dengan Upload Gambar**

Setelah kita mengetahui hasil data tersebut berdasarkan akurasi pada model data. Maka, kita bisa gunakan "upload image" dari Google Colab untuk memprediksi model data yang telah di klasifikasi.

In [ ]:
#Memprediksi Model Data yang ada dengan Upload Gambar

import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():

    path = fn
    img = image.load_img(path, target_size=(100,100))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
 
    images = np.vstack([x])
    classes = model.predict(images, batch_size=32)
  
    print(fn)
    if classes[0,0] == 1.0:
      print('Paper')
    elif classes[0,1] == 1.0:
      print('Rock')
    else:
      print('Scissors')
